# Ask

For the 10k pids:
- what other data types they have, E.g  EHR, 'The Basics', 'Lifestyle', 'Overall Health', 'Personal and Family Health History', 'Social Determinants of Health' surveys, etc.  

-  deep dive EHR and characterize it and evaluate how comprehensive it is. May be we can simply populate a figure like this “historical availability of EHR records (?)

# Set Up

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from postgres_config import db_postgres
import os
import psycopg2
from google.cloud import bigquery
client = bigquery.Client()
v7_dataset  ='aou-res-curation-output-prod.C2022Q4R11'
#cloud_sql_proxy -instances=aou-pdr-data-prod:us-central1:prod-pdr-5deb-lhty=tcp:7000,aou-pdr-data-prod:us-central1:prod-pdr-alpha-replica=tcp:7005

In [ ]:
allv7_pids = pd.read_gbq(f'''
    SELECT DISTINCT p.person_id as research_id
    , has_ppi_survey_data AS PPI             
    , has_physical_measurement_data AS PM 
    , has_ehr_data AS EHR
    , has_fitbit AS Fitbit                
    , has_whole_genome_variant AS WGS
    , has_array_data as Arr
    , has_lr_whole_genome_variant as lr_wgs
    , has_structural_variant_data as strutural_variant
    FROM `{v7_dataset}.person` p
    LEFT JOIN `{v7_dataset}.cb_search_person` USING(person_id) ''')

display(allv7_pids.head(2))
allv7_pids.to_csv('v7_ct_participant.csv')

In [ ]:
filename = 'data_characterization_multi-omics_pids.xlsx'
writer = pd.ExcelWriter(filename)

## mapping the 10 pids : biobank_id --> person_id

In [ ]:
the_10k = pd.read_excel('the_10k_mar2024.xlsx')[['biobank_id']]
the_10k['biobank_id'] = [int(p.replace('A','')) for p in the_10k['biobank_id']]
the_10k.head(2)

In [ ]:
bio_pid_map = pd.read_sql('''SELECT DISTINCT research_id as person_id, biobank_id, participant_id FROM pdr.mv_participant''', db_postgres)
v7_pids = pd.read_gbq(f'''SELECT DISTINCT person_id FROM `{v7_dataset}.person`''')

In [ ]:
the10k_mapped = the_10k.merge(bio_pid_map)#.merge(v7_pids)
the10k_mapped.head(2)

In [ ]:
set(the10k_mapped.person_id) in set(v7_pids.person_id)\
, set(the10k_mapped.person_id) == set(the10k_mapped.merge(v7_pids).person_id)\
, the10k_mapped.person_id.nunique(), the_10k.biobank_id.nunique()

In [ ]:
len(set(the10k_mapped.person_id) - set(v7_pids.person_id))

In [ ]:
## only those of the 10 k that are in v7 (503 bids are not in v7)
v7_10k_mapped = the10k_mapped.merge(v7_pids)
v7_10k_mapped.person_id.nunique()

In [ ]:
v7_10k_mapped_tp = tuple(v7_10k_mapped.person_id.unique())

# What dtypes do they have

In [ ]:
#%%writefile overall_summary.py
from google.cloud import bigquery
client = bigquery.Client()
import os
import subprocess
import pandas as pd
import numpy as np
from datetime import datetime

# For Data Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from venn import venn
#from utilities import utilities as u

class overall_summary:    
    ## Class to get ubr flags for each person in the current dataset    
    def __init__(self):
        self.dataset = 'aou-res-curation-output-prod.C2022Q4R11'
        
    def client_read_gbq(self, query, jobconfig = None):
        # function to read data from BQ into py dataframe with the client
        if jobconfig is None: job_config = bigquery.QueryJobConfig(default_dataset=self.dataset)
        else: job_config = jobconfig
        query_job = client.query(query, job_config =job_config)  # API request
        df = query_job.result().to_dataframe()
        return df
        
#     def aian_pids(self):
#         aian_pids = self.client_read_gbq(f'''SELECT DISTINCT person_id FROM `ds_survey`
#                                 WHERE question_concept_id = 1586140 AND answer = 'What Race Ethnicity: AIAN' ''')
#         return aian_pids
    
#     def wear_pids(self):
#         wear_pids = self.client_read_gbq(f'''SELECT DISTINCT person_id 
#                                         FROM `wear_study` WHERE resultsconsent_wear= 'Yes' ''')
#         return wear_pids

    def data_types(self, pids):
    
        # Query to get a list of all participants and their data types
        data_types_df = self.client_read_gbq(f"""
            SELECT 
                DISTINCT p.person_id
                , has_ppi_survey_data AS PPI             
                , has_physical_measurement_data AS PM 
                , has_ehr_data AS EHR
                , has_fitbit AS Fitbit                
                , has_whole_genome_variant AS WGS
                , has_array_data as Arr
                ---, has_lr_whole_genome_variant as lr_wgs
                --, has_structural_variant_data as strutural_variant
                --, CASE WHEN has_whole_genome_variant = 1 OR has_array_data = 1 THEN 1 END AS has_wgs_or_array
                --, CASE WHEN has_whole_genome_variant = 1 AND has_array_data = 1 THEN 1 END AS has_wgs_and_array

            FROM `person` p
            LEFT JOIN `cb_search_person` USING(person_id) 
            WHERE person_id IN {pids}
            
            """)
        data_types_df = data_types_df.rename(columns = {'Arr':"Array"})
        data_types_df = data_types_df.fillna(0).astype('int64')

        return data_types_df
    
    def add_on_data_types(self, pids):
    
        # Query to get a list of all participants and their data types
        data_types_df = self.client_read_gbq(f"""
            SELECT 
                DISTINCT p.person_id
                , has_lr_whole_genome_variant as lr_wgs
                , has_structural_variant_data as strutural_variant
                , CASE WHEN has_whole_genome_variant = 1 OR has_array_data = 1 THEN 1 END AS has_wgs_or_array
                , CASE WHEN has_whole_genome_variant = 1 AND has_array_data = 1 THEN 1 END AS has_wgs_and_array

            FROM `person` p
            JOIN `cb_search_person` USING(person_id) 
            WHERE person_id IN {pids}""")
        data_types_df = data_types_df.fillna(0).astype('int64')

        return data_types_df

    def calc_percentage(self, df, denom):
        df['% of total participants in CDR'] = df['Count of Participants in CDR']/denom
        df['% of total participants in CDR'] = ['{:.2%}'.format(i) for i in df['% of total participants in CDR']]

        return df

    def count_by_data_type(self, df):
        #display(data_types_df)
        denom = df.person_id.nunique()
        
        DF = df.melt(id_vars = ['person_id'])
        DF = DF[DF.value == 1].drop('value', axis =1)

        DF_none = df.loc[df[df.drop('person_id', axis =1).columns].sum(axis = 1) == 0][['person_id']]
        DF_none['variable'] = 'None'

        DF = pd.concat([DF, DF_none])

        DF_count = DF.groupby(['variable'], as_index = False).nunique().sort_values('person_id')
        DF_count.columns = ['Data Type','Count of Participants in CDR']

        DF_count = self.calc_percentage(DF_count, denom = denom)\
            .rename(columns = {'Count of Participants in CDR':'Count of Participants in CDR ('+'{:,}'.format(denom)+')'})

        return DF_count


    def count_by_n_data_type(self, df):
        
        denom = df.person_id.nunique()
        n_data_types_df = pd.DataFrame(df.set_index('person_id').sum(axis = 1)
                                      , columns = ['n_data_types']).reset_index()


        ## By number of data type
        DF_count = n_data_types_df.groupby(['n_data_types'], as_index = False).nunique()
        DF_count.columns = ['Number of Data Types', 'Count of Participants in CDR']

        max_n = int(DF_count['Number of Data Types'].max())
        DF_count['Number of Data Types'] = ['Participants with \nONLY '+str(int(i))+' Data Type(s)' \
                                            for i in DF_count['Number of Data Types']]   
        DF_count = pd.concat([DF_count
                              , pd.DataFrame({'Number of Data Types': 'TOTAL PARTICIPANTS IN CDR'
                                              , 'Count of Participants in CDR':DF_count['Count of Participants in CDR'].sum()}
                                             , index = [''])])

        DF_count = self.calc_percentage(DF_count, denom = denom)
        DF_count['Number of Data Types'] = DF_count['Number of Data Types']\
                                                .replace({'Participants with \nONLY 0 Data Type(s)':'Participants with \n0 Data Type(s)'
                                                 , 'Participants with \nONLY '+str(max_n)+' Data Type(s)':'Participants with \nall '\
                                                  +str(max_n)+' Data Type(s)'
                                                     })

        ## By number and type of Data
        DF_none = n_data_types_df[n_data_types_df.n_data_types ==0]
        DF_none['data_types'] = 'None'

        DF = df.melt(id_vars = ['person_id'])
        DF = DF[DF.value ==1].drop('value', axis =1)
        DF = DF.merge(n_data_types_df[n_data_types_df.n_data_types >0])

        DF['data_types'] = DF.groupby(['person_id','n_data_types'])['variable'].transform(lambda x: ' and '.join(x))
        DF = DF[['person_id','data_types']].merge(n_data_types_df[n_data_types_df.n_data_types >0])

        DF_count_d = pd.concat([DF, DF_none])
        DF_count_d = DF_count_d.groupby(['n_data_types','data_types'], as_index = False).nunique()
        DF_count_d.columns = ['Number of Data Types','Data Types', 'Count of Participants in CDR']


        DF_count_d['Number of Data Types'] = ['Participants with ONLY '+str(int(i))+' Data Type(s)' for i in DF_count_d['Number of Data Types']]

        DF_count_d = pd.concat([DF_count_d
                                , pd.DataFrame({
                                               'Number of Data Types': ' '
                                               , 'Count of Participants in CDR':DF_count_d['Count of Participants in CDR'].sum()
                                               , 'Data Types': 'TOTAL PARTICIPANTS IN CDR'
                                               }, index = [''])])

        DF_count_d = self.calc_percentage(DF_count_d, denom = denom)

        DF_count_d['Number of Data Types'] = DF_count_d['Number of Data Types']\
                        .replace({'Participants with ONLY 0 Data Type(s)':'Participants with 0 Data Types'
                                 , 'Participants with ONLY '+str(max_n)+' Data Type(s)':'Participants with all '\
                                  +str(max_n)+' Data Type(s)'
                                 })
        DF_count_d['Count of Participants in CDR'] = DF_count_d['Count of Participants in CDR'].astype('int')

        return DF_count, DF_count_d

    def format_numbers(self, df):
        # format the counts and percentage columns

        formated_df = df.copy()
        for col in formated_df.columns:
            if formated_df[col].dtype == 'int64':
                formated_df[col] = ['{:,}'.format(i) for i in formated_df[col]]

        return formated_df

    def combine_count_and_perc(self, df, count_perc_cols_dic, drop = True):    
        df_comb = df.copy()

        for count_col in count_perc_cols_dic:
            perc_col = count_perc_cols_dic[count_col]
            df_comb[count_col+'_str'] = df_comb[count_col]
            df_comb[perc_col+'_str'] = df_comb[perc_col]
            df_comb[[count_col+'_str', perc_col+'_str']] = self.format_numbers(df_comb[[count_col+'_str', perc_col+'_str']])
            new_col_name = count_col+' (and %)'
            df_comb[new_col_name] = df_comb[count_col+'_str'].astype('str')+' ('+df_comb[perc_col+'_str'].astype('str')+')'

            if drop == True:
                df_comb = df_comb.drop([count_col, count_col+'_str', perc_col, perc_col+'_str'], axis =1)
        return df_comb

    def format_dataframe(self, df):

        display_DF = pd.DataFrame()
        for n in df['Number of Data Types'].unique():
            DF = df[df['Number of Data Types'] == n].drop('Number of Data Types', axis =1)
            DF = DF.reindex([n]+ DF.index.tolist()).fillna('')
            display_DF = pd.concat([display_DF, DF])

        index_list = display_DF.index.tolist()
        index_list = [i for i in index_list if type(i) == int]

        dic = {0:''}
        for i in index_list:
            dic.update({i:''})
        display_DF = display_DF.rename(index=dic)
        return display_DF


    def combine_final_tables(self, dfs_to_combine, indexname, save = True):
        table = pd.DataFrame()
        for colname, df in dfs_to_combine.items():
            DF = df.set_index(indexname)
            DF.columns = [colname]
            table = pd.concat([table,DF],axis =1).fillna('-')
            
                ###################### save the file to the bucket
        if save == True:
            add = indexname[0].lower().replace(' ', '_')
            filename = f'1_count_by_{add}.xlsx'
            u().write_to_csv_excel(table, filename = filename)
            #print(f'Saving the file to the bucket as {filename}')
            #table.to_excel(filename)
            #args = ["gsutil", "cp", f"./{filename}", f"{self.bucket}/notebooks/"]
            #output = subprocess.run(args, capture_output=True)
            #print(output.stderr)

        return table
    
    def table_1_by_pop(self, pop_df):
        n = pop_df.person_id.nunique()
        table1_1_uncomb = self.count_by_data_type(df = pop_df)
        table1_2_uncomb, table1_2_raw = self.count_by_n_data_type(df = pop_df)

        count_col = "Count of Participants in CDR"; perc_col = "% of total participants in CDR"
        table1_1 = self.combine_count_and_perc(table1_1_uncomb
                                        , count_perc_cols_dic= dict({f"{count_col} ({'{:,}'.format(n)})":f'{perc_col}'}))
        table1_2 = self.combine_count_and_perc(table1_2_raw, count_perc_cols_dic= dict({f'{count_col}':f'{perc_col}'}))
        table1_2 = table1_2.reset_index(drop = True)

        return table1_1, table1_2, table1_1_uncomb, table1_2_uncomb

    def venn_plot(self, data_types_df, pid_type, figname, genomics = 'no'):
        ppi_set = set(data_types_df[data_types_df.PPI == 1].person_id)
        ehr_set = set(data_types_df[data_types_df.EHR == 1].person_id)
        pm_set = set(data_types_df[data_types_df.PM == 1].person_id)
        fitbit_set = set(data_types_df[data_types_df.Fitbit == 1].person_id)

        data = {'PPI': ppi_set
            , 'EHR': ehr_set#.intersection(ppi_1_3_set)
            , 'Phys. Meas.': pm_set
            , 'Fitbit': fitbit_set}

        if genomics.lower() == 'yes':
            genomics_set = set(data_types_df[(data_types_df.Array == 1) | (data_types_df.WGS ==1)].person_id)
            data.update({'Genomics': genomics_set})

        v = venn(data)
        plt.title(f'Count of {pid_type} participants with multiple data types \n', fontsize = 18)
        plt.tight_layout()
        plt.savefig(f'{figname}.jpeg')

    def combine_plot_data(self, xcol, dfs_to_combine):

        count_col = "Count of Participants in CDR"
        perc_col = "% of total participants in CDR" 

        ## whole cdr
        kw = 'Whole CDR'
        df_cdr_raw = dfs_to_combine[kw]
        df_cdr = df_cdr_raw.copy()
        df_cdr['Participant'] = kw
        df_cdr = df_cdr.rename(columns = {f"{[c for c in df_cdr.columns if 'Count of Participants' in c][0]}":f'{count_col}'})


        DF = pd.DataFrame()
        for key in [k for k in dfs_to_combine.keys() if k != 'Whole CDR']:
            df = dfs_to_combine[key]
            n = df.iloc[-1,1]
            df['Number of Data Types'] = df[xcol]#.replace('Participants with \nall 5 Data Type(s)','Participants with \nONLY 5 Data Type(s)')
            rename_c = [c for c in df.columns if f'{count_col}' in c][0]
            df = df.rename(columns = {rename_c:f'{count_col}'})

            missing_rows = set(df_cdr[xcol]) - set(df[xcol])
            none = pd.DataFrame(missing_rows, columns = [xcol])
            none[f'{count_col}'] = 0
            none[f'{perc_col}'] = '0%'
            none[xcol] = 'Participants with \n0 Data Type(s)'

            df = pd.concat([none, df]).reset_index(drop = True)
            df['Participant'] = key
            DF = pd.concat([DF, df]).drop_duplicates()

        DF = pd.concat([df_cdr, DF]).drop_duplicates()       
        return DF

    def create_plot_label(self, df_plot, count_col ='Count of Participants in CDR'
                         , perc_col = '% of total participants in CDR'):
        df_plot_final = df_plot.drop_duplicates().reset_index(drop = True)

        df_plot_final[perc_col] = [round(float(c.replace('%', ''))) for c in df_plot_final[perc_col]]
        df_plot_final['label'] = ['{:,}'.format(i) for i in df_plot_final[count_col]] 
        df_plot_final['label'] = df_plot_final['label']+ ' ('+ df_plot_final[perc_col].astype('str')+ '%)'
        df_plot_final['label'] = [c.replace(' (', '\n(') for c in df_plot_final['label']]
        return df_plot_final

    def bar_plot(self, df, title, label, y, x= 'Data Type', hue= 'Participant'
                     , w = 14, h = 10, r = 0, ha = 'left', va = 'bottom'):


        plt.figure(figsize=(w,h), tight_layout=True)

        ax = sns.barplot(x= x, y = y
                             , hue = hue
                             , palette='Blues', ci=None
                             , edgecolor = 'w', data = df)

        for p in ax.patches:
            n = int(p.get_height())
            l = df.loc[df[y] == n, label].reset_index(drop = True)[0]

            t = ax.annotate(l, xy = (p.get_x(), n))
            t.set(color = "black", size = 12, rotation=r, horizontalalignment= ha, verticalalignment=va)

        ax.set(xlabel="",ylabel="")
        ax.set_title(title+'\n\n', fontsize=18)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=r, horizontalalignment= 'center', fontsize=12)
        ax.set(yticklabels=[])

        plt.savefig(title.replace('Count of Participants in the CDR ', 'Barplot ')+'.jpeg')
        plt.show()

In [ ]:
# Input Data
s = overall_summary()
data_types_df = s.data_types(pids = v7_10k_mapped_tp)
data_types_df.head()

In [ ]:
data_types_df[(data_types_df.Array == 1) | (data_types_df.WGS == 1)].person_id.nunique()

In [ ]:
add_data_types_df = s.add_on_data_types(pids = v7_10k_mapped_tp)
add_data_types_df.head()

In [ ]:
add_data_types_df.columns

In [ ]:
#add_data_types_df[(add_data_types_df.has_wgs_or_array == 1) | (add_data_types_df.strutural_variant == 1) | (add_data_types_df.lr_wgs == 1)].person_id.nunique()

In [ ]:
## Deliverable table 1
sum_reports_cdr = s.table_1_by_pop(data_types_df)
sum_reports_cdr[0]

In [ ]:
sum_reports_cdr[0].to_excel(writer, 'Data Availability')

In [ ]:
## Deliverable plot 1
s.venn_plot(data_types_df, pid_type = "CDR",figname = 'WholeCDR_gen_Venn', genomics= 'yes')

# EHR data characterization

In [ ]:
ehr_query= f"""

WITH ehr AS (
    SELECT DISTINCT person_id, 1 as has_ehr, 'measurement' as domain
    FROM `measurement` 
    JOIN `measurement_ext` USING(measurement_id)
    WHERE LOWER(src_id) LIKE 'ehr site%'

    UNION DISTINCT
    SELECT DISTINCT person_id, 1 as has_ehr, 'condition' as domain
    FROM `condition_occurrence` 
    JOIN `condition_occurrence_ext` USING(condition_occurrence_id)
    WHERE LOWER(src_id) LIKE 'ehr site%'

    UNION DISTINCT
    SELECT DISTINCT person_id, 1 as has_ehr, 'device' as domain
    FROM `device_exposure` 
    JOIN `device_exposure_ext` USING(device_exposure_id)
    WHERE LOWER(src_id) LIKE 'ehr site%'

    UNION DISTINCT
    SELECT DISTINCT person_id, 1 as has_ehr, 'drug' as domain
    FROM `drug_exposure` 
    JOIN `drug_exposure_ext` USING(drug_exposure_id)
    WHERE LOWER(src_id) LIKE 'ehr site%'

    UNION DISTINCT
    SELECT DISTINCT person_id, 1 as has_ehr, 'observation' as domain
    FROM `observation` 
    JOIN `observation_ext` USING(observation_id)
    WHERE LOWER(src_id) LIKE 'ehr site%'

    UNION DISTINCT
    SELECT DISTINCT person_id, 1 as has_ehr, 'procedure' as domain
    FROM `procedure_occurrence` 
    JOIN `procedure_occurrence_ext` USING(procedure_occurrence_id)
    WHERE LOWER(src_id) LIKE 'ehr site%' 

    UNION DISTINCT
    SELECT DISTINCT person_id, 1 as has_ehr, 'visit' as domain
    FROM `visit_occurrence` 
    JOIN `visit_occurrence_ext` USING(visit_occurrence_id)
    WHERE LOWER(src_id) LIKE 'ehr site%'
)

SELECT DISTINCT ehr.* 
FROM ehr
WHERE person_id IN {v7_10k_mapped_tp}

"""

ehr_df = s.client_read_gbq(ehr_query)
ehr_df.head(2)

In [ ]:
no_ehr = set(v7_10k_mapped.person_id) - set(ehr_df.person_id)
len(no_ehr), ehr_df.person_id.nunique(), data_types_df[data_types_df.person_id.isin(no_ehr)].EHR.unique()

In [ ]:
ehr_df.person_id.nunique(), ehr_df.person_id.nunique()+1566

----------------

In [ ]:
ehr_df.domain.unique()

In [ ]:
ehr_pv_df = ehr_df.pivot(index=['person_id'], columns = ['domain']).fillna(0).reset_index()
ehr_pv_df.columns = [c[1] for c in ehr_pv_df.columns]
ehr_pv_df = ehr_pv_df.rename(columns = {ehr_pv_df.columns[0]:'person_id'})
ehr_pv_bin_df = pd.concat([ehr_pv_df
                          , data_types_df[data_types_df.person_id.isin(no_ehr)][['person_id']].drop_duplicates()
                          ]).fillna(0)
ehr_pv_bin_df          

In [ ]:
ehr_sum_reports_cdr = s.table_1_by_pop(ehr_pv_bin_df)

In [ ]:
85+15

In [ ]:
### Deliverable table 2
ehr_sum_reports_cdr[0]

In [ ]:
ehr_sum_reports_cdr[0].to_excel(writer, 'Data Availability_EHR')

In [ ]:
sum_reports_cdr2 = ehr_sum_reports_cdr[1]
sum_reports_cdr2['Number of Data Types'] = [i.replace('Participants with ', '').replace('Data Type', 'EHR Domain')\
                                            .replace('0 EHR Domains','No EHR Data')
                                            for i in sum_reports_cdr2['Number of Data Types']]
sum_reports_cdr2 = sum_reports_cdr2.set_index(['Number of Data Types', 'Data Types'])
sum_reports_cdr2

In [ ]:
sum_reports_cdr3 = ehr_sum_reports_cdr[3]
sum_reports_cdr3['Number of Data Types'] = [i.replace('Participants with ', '').replace('Data Type', 'EHR Domain')\
                                            .replace('0 EHR Domains','No EHR Data')
                                            for i in sum_reports_cdr3['Number of Data Types']]
sum_reports_cdr3 = sum_reports_cdr3.set_index(['Number of Data Types'])
#sum_reports_cdr3.to_excel('add.xlsx')

In [ ]:
sum_reports_cdr3.to_excel(writer, 'EHR Characterization')
sum_reports_cdr2.to_excel(writer, 'EHR Characterization - Details')

writer.save()

## historic ehr

In [ ]:
# For Data Manipulation
import numpy as np
import os
import pandas as pd
import math
import statistics
import datetime 
# For Data Visualization
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
#import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

class historic_ehr:

    def __init__(self):
        self.dataset = 'aou-res-curation-output-prod.C2022Q4R11'
        self.pids = v7_10k_mapped_tp
        
    def client_read_gbq(self, query, jobconfig = None):
        # function to read data from BQ into py dataframe with the client
        if jobconfig is None: job_config = bigquery.QueryJobConfig(default_dataset=self.dataset)
        else: job_config = jobconfig
        query_job = client.query(query, job_config =job_config)  # API request
        df = query_job.result().to_dataframe()
        return df
        
    def ehr_by_domain(self, domain, start_date_field, end_date_field, table, table_id, cutoff):

        concept_id = domain.lower()+'_concept_id'
            #print(domain, start_date_field, end_date_field, table, table_id, concept_id)
        df = self.client_read_gbq(f'''
                SELECT DISTINCT person_id, '{domain}' as ehr_domain, src_id AS ehr_site
                , vocabulary_id as vocabulary, MIN({start_date_field}) AS start_date
                , CASE WHEN MAX({end_date_field}) IS NULL THEN "{cutoff}" ELSE MAX({end_date_field}) END AS end_date

                FROM `{table}`
                JOIN `{table}_ext` USING({table_id})
                JOIN `concept` on concept_id = {concept_id}
                WHERE LOWER(src_id) LIKE 'ehr site%' AND person_id in {self.pids}
                GROUP BY 1,2,3,4''')
        return df
    
    def historic_ehr_data(self):
        # setting variables
        tables = ['measurement', 'condition_occurrence','device_exposure','drug_exposure'
                      ,'observation','procedure_occurrence', 'visit_occurrence']
        start_end_date_fields = [['measurement_date', 'measurement_date']
                                     , ['condition_start_date','condition_end_date']
                                     , ['device_exposure_start_date','device_exposure_end_date']
                                     , ['drug_exposure_start_date','drug_exposure_end_date']
                                     , ['observation_date','observation_date']
                                     , ['procedure_date','procedure_date']
                                     , ['visit_start_date','visit_end_date']
                                    ]
        domains = ['Measurement', 'Condition','Device','Drug', 'Observation','Procedure', 'Visit']
        table_ids = ['measurement_id', 'condition_occurrence_id','device_exposure_id', 'drug_exposure_id'
                         ,'observation_id', 'procedure_occurrence_id','visit_occurrence_id']


        ehr_cutoff_date = self.client_read_gbq(f'''SELECT ehr_cutoff_date  FROM `._cdr_metadata`''')
        ehr_cutoff =str(ehr_cutoff_date.ehr_cutoff_date[0])   

        historic_ehr_raw = pd.DataFrame()

        for i in range(len(tables)):
            df = self.ehr_by_domain(domain = domains[i]
                                    , start_date_field = start_end_date_fields[i][0]+'time'
                                    , end_date_field = start_end_date_fields[i][1]+'time'
                                    #, dataset = self.dataset
                                    , table = tables[i], table_id = table_ids[i]
                                    , cutoff = ehr_cutoff)
            historic_ehr_raw = pd.concat([historic_ehr_raw, df])

        historic_overall = historic_ehr_raw.copy()
        historic_overall['ehr_domain'] = 'Any EHR'
        historic_overall = historic_overall.drop_duplicates()#.groupby(['person_id','ehr_domain'], as_index= False).min()
        historic_ehr = pd.concat([historic_ehr_raw, historic_overall])

        return historic_ehr

class historic_ehr_summary:

    def __init__(self, denominator, historic_ehr_df):
        self.denominator = denominator
        self.historic_ehr_df = historic_ehr_df
             
    def years_of_data(self, historic_ehr_df):
        ## Historic EHR data by Domains over time, dataframe 
        df = historic_ehr_df.reset_index(drop = True)
        df = df[['person_id','ehr_domain','start_date', 'end_date']].drop_duplicates()
        ###
        end_date_max = df[['person_id','ehr_domain','end_date']].drop_duplicates()\
                            .groupby(['person_id','ehr_domain'], as_index = False).max()

        start_date_min = df[['person_id','ehr_domain','start_date']].drop_duplicates()\
                            .groupby(['person_id','ehr_domain'], as_index = False).min()

        df1 = df.drop(['start_date','end_date'],1).merge(start_date_min, 'left')\
                                .merge(end_date_max, 'left').drop_duplicates()

        df1['days_diff'] = df1['end_date'] - df1['start_date']
        df1['days_diff'] = [abs(i.days) for i in df1['days_diff']] 


        df1.loc[df1['days_diff'] == 0, 'days_diff'] = 1
        
        ###
        df_years_of_data = df1[['person_id','ehr_domain','days_diff']]\
                                .groupby(['person_id','ehr_domain'], as_index = False).sum()
        df_years_of_data['years_of_data'] = df_years_of_data['days_diff']/365
        df_years_of_data = df_years_of_data.sort_values('years_of_data')
        years_of_data_per_pid = df_years_of_data.drop('days_diff',1)
        
        return years_of_data_per_pid
    

    def pid_cumcount_by_vars(self, df, cum_count_by):

        cummulative_counts_df = pd.DataFrame()

        for d in df[cum_count_by[0]].unique():
            #print('\n'+d)
            DF = df[df[cum_count_by[0]] == d][['person_id']+cum_count_by].sort_values(cum_count_by, ascending = True)
            DF['cumcount'] = (~DF['person_id'].duplicated()).cumsum()
            cum_df = DF.drop('person_id',1).drop_duplicates().reset_index(drop = True)
            cum_df[cum_count_by[0]] = d
            cum_df = cum_df.sort_values(cum_count_by).groupby(cum_count_by, as_index = False).last() 

            cummulative_counts_df = pd.concat([cummulative_counts_df, cum_df])

        cummulative_counts_df['cumcount'] = cummulative_counts_df['cumcount'].astype('int')

        return cummulative_counts_df

    ###############################    
        
    def stats_df(self, years_of_data_per_pid):
        historic_ehr_years_stats = years_of_data_per_pid.groupby(['ehr_domain']).agg({'person_id':'nunique', 'years_of_data'\
                                                                      :['mean','median', 'min','max','std']})

        historic_ehr_years_stats.columns = [c[1] for c in historic_ehr_years_stats.columns]
        historic_ehr_years_stats[['mean','median','min','max', 'std']] = \
        historic_ehr_years_stats[['mean','median','min','max', 'std']].apply(lambda x: round(x,2)).astype('float64')

        historic_ehr_years_stats[['nunique','mean','median','min','max', 'std']] = \
                            format_numbers(historic_ehr_years_stats[['nunique','mean','median','min','max', 'std']])
        historic_ehr_years_stats['min'] = historic_ehr_years_stats['min'].replace('0%','0')

        name = 'Years of EHR Data Available'
        historic_ehr_years_stats.columns = pd.MultiIndex.from_tuples(
                                                    [('','N Participants')
                                                    ,(f"{name}",'Mean')
                                                    ,(f"{name}",'Median')
                                                    ,(f"{name}",'Minimum')
                                                    ,(f"{name}",'Maximun')
                                                    ,(f"{name}",'Standard Deviation')
                                                    ])

        return style_df(historic_ehr_years_stats)

    def boxplot(self, years_of_data_per_pid, w = 16, l = 8):

        df_plot = years_of_data_per_pid.groupby(['ehr_domain','years_of_data'], as_index = False).nunique()
        current_cdr = dataset.split('.')[1]       
 
        sns.set(rc={'figure.figsize':(w, l)})
        g = sns.boxplot(data = df_plot
                            , x = 'ehr_domain', y = 'years_of_data', hue = 'ehr_domain'
                            , medianprops={"color": "red"}, meanprops={"color": "coral"}
                            , showmeans = True, dodge=False)

        g.get_legend().remove()

        plt.suptitle(f'''\nIn the ({current_cdr}) for N Participants = {'{:,}'.format(self.denominator)}\n'''
                         , size = 16,style='italic')
        plt.title('\nYears of EHR Data Available\n\n\n', size = 20)
        plt.xlabel('EHR Domains', size = 16)
        plt.ylabel('N Years of Data', size = 16)
        plt.xticks(size = 14)
        plt.yticks(size = 14)

        plt.show()                
    
    ############################# 2. Historic EHR data by Domains over time, plot #############################            
    def lineplot(self, cdr_version, add_perc_text = 'yes', rotate_x = 0, plot_start_year = 2010#, save_plot = 'yes'
                , w = 18, l = 14):
 
        #Transform
        historic_ehr_cum = self.historic_ehr_df.copy()
        historic_ehr_cum['start_year'] = [i.year for i in historic_ehr_cum['start_date']]
        historic_ehr_cum = self.pid_cumcount_by_vars(df  = historic_ehr_cum[['person_id','ehr_domain','start_year']].drop_duplicates()
                                                , cum_count_by = ['ehr_domain','start_year'])
        historic_ehr_cum['cum%'] = round(historic_ehr_cum['cumcount']*100/self.denominator)
        historic_ehr_cum['cum%'] = [int(i) for i in historic_ehr_cum['cum%']] #new added 05/14/2024
        historic_ehr_cum = historic_ehr_cum[historic_ehr_cum['cum%'] > 0] #new added 12/7/2023

        historic_ehr_cum_afterYEAR = historic_ehr_cum.copy()
        historic_ehr_cum_afterYEAR = historic_ehr_cum_afterYEAR[historic_ehr_cum_afterYEAR.start_year >= plot_start_year]

        df_plot = historic_ehr_cum_afterYEAR.drop(['cumcount'],1)\
                    .pivot(index = ['start_year'], columns = ['ehr_domain']).fillna(0)#.reset_index()
        

        df_plot.columns = [c[1] for c in df_plot.columns]

        #current_cdr = self.dataset.split('.')[1]
#         fig_caption = f'''\n\nThis plot represents EHR data availability in the current CDR ({current_cdr}). 
#         The denominator for the percentages is the Total Number of Participants in the current CDR, N = {'{:,}'.format(self.denominator)}.
#         For better visibility, the plot only displays EHR data growth starting in 2010. '''
        fig_caption = f'''
\n\n\nThe denominator for the percentages is the Total Number of Participants, N = {'{:,}'.format(self.denominator)}.
For better visibility, the plot only displays EHR data growth starting in {plot_start_year}.
Dataset version: {cdr_version}'''
    
        #plot
        plt.figure(figsize=(w,l), tight_layout=True, facecolor="w")       
        plt.plot(df_plot, 'o-', linewidth=2)
        

        plt.xticks(df_plot.index, size = 13, rotation = rotate_x)
        plt.yticks(range(int(historic_ehr_cum_afterYEAR['cum%'].min())
                         , int(historic_ehr_cum_afterYEAR['cum%'].max()), 5),size = 13)
        plt.gca().yaxis.set_major_formatter(PercentFormatter(decimals=0))
        plt.xlabel('Year of EHR Data\n\n\n\n', size = 16)
        plt.ylabel('% of Participants with EHR data', size = 16)
        plt.title('\nHistorical Availability of EHR Data', size = 21)
        plt.xticks(size = 14, rotation=45)
        plt.yticks(size = 14)
        plt.legend(title='EHR Domains', title_fontsize = 14, labels=df_plot.columns, fontsize = 14)
        plt.text(statistics.median(df_plot.index), -16
             , fig_caption, verticalalignment='bottom',style='italic'
             , horizontalalignment='center', color = 'black', size = 14)

        if add_perc_text.lower() == 'yes':
            min_year = df_plot.index.min()
            max_year = df_plot.index.max()

            for d in ['Any EHR']:
                plt.text(min_year+0.5, df_plot[d][min_year]+2
                         , d+' ('+str(int(df_plot[d][min_year]))+'%)',verticalalignment='center'
                         ,horizontalalignment='right', color = 'black', size = 12, rotation = 13)
                plt.text(max_year-0.0001, df_plot[d][max_year]+1
                         , d+' ('+str(int(df_plot[d][max_year]))+'%)',verticalalignment='bottom'
                         ,horizontalalignment='center', color = 'black', size = 12)

            for d in ['Condition', 'Device', 'Drug'#,'Measurement'
                      ,'Observation'#,'Procedure'
                      ,'Visit']:
                plt.text(min_year-0.1, df_plot[d][min_year]-0.6
                         , str(int(df_plot[d][min_year]))+'%',verticalalignment='center'
                         ,horizontalalignment='right', color = 'black', size = 12, rotation = 13)

            for d in ['Condition', 'Device', 'Drug'#,'Measurement'
                      ,'Observation'#,'Procedure','Visit'
                     ]:
                plt.text(max_year+0.3, df_plot[d][max_year]-1
                         , str(int(df_plot[d][max_year]))+'%',verticalalignment='bottom'
                         ,horizontalalignment='center', color = 'black', size = 12)
    
        plt.savefig(f'ehr_historical_availability_from_{plot_start_year}_.jpeg')
        plt.show()


In [ ]:
h = historic_ehr()
historic_ehr_df = h.historic_ehr_data()
historic_ehr_df.head()

In [ ]:
denominator = v7_10k_mapped.person_id.nunique()

hs = historic_ehr_summary(denominator, historic_ehr_df)
hs.lineplot(cdr_version = 'V7 Controlled Tier')